<a href="https://colab.research.google.com/github/Cobord/Quantum-Computer-Things/blob/master/Information%20Geometry/geometricMeanKnutsonTao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import timeit
import sys
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Found GPU at: /device:GPU:0


In [16]:
eps_default=1e-10
matrix_dim_default=500

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu(matrix_dim=matrix_dim_default,eps=eps_default,to_print=False):
  with tf.device('/cpu:0'):
    a_matrix_helperC = tf.random_normal((matrix_dim, matrix_dim))
    a_matrixC=tf.matmul(tf.linalg.adjoint(a_matrix_helperC),a_matrix_helperC)
    b_matrix_helperC = tf.random_normal((matrix_dim, matrix_dim))
    b_matrixC=tf.matmul(tf.linalg.adjoint(b_matrix_helperC),b_matrix_helperC)
    matC=tf.matmul(tf.linalg.inv(a_matrixC),b_matrixC)
    sC, uC, vC = tf.svd(matC)
    siC = tf.where(tf.less(sC, eps), sC, tf.sqrt(sC))
    abgm_helperC=tf.matmul(tf.matmul(uC, tf.diag(siC)), vC, transpose_b=True)
    a_b_geometric_meanC=tf.matmul(a_matrixC,abgm_helperC)
    all_eigsC=tf.stack([tf.linalg.eigvalsh(a_matrixC),tf.linalg.eigvalsh(b_matrixC),tf.linalg.eigvalsh(a_b_geometric_meanC)])
    if(to_print):
      print(sess.run(all_eigsC))
    else:
      sess.run(all_eigsC)
  
def gpu(matrix_dim=matrix_dim_default,eps=eps_default,to_print=False):
  with tf.device('/gpu:0'):
    a_matrix_helperG = tf.random_normal((matrix_dim, matrix_dim))
    a_matrixG=tf.matmul(tf.linalg.adjoint(a_matrix_helperG),a_matrix_helperG)
    b_matrix_helperG = tf.random_normal((matrix_dim, matrix_dim))
    b_matrixG=tf.matmul(tf.linalg.adjoint(b_matrix_helperG),b_matrix_helperG)
    matG=tf.matmul(tf.linalg.inv(a_matrixG),b_matrixG)
    sG, uG, vG = tf.svd(matG)
    siG = tf.where(tf.less(sG, eps), sG, tf.sqrt(sG))
    abgm_helperG=tf.matmul(tf.matmul(uG, tf.diag(siG)), vG, transpose_b=True)
    a_b_geometric_meanG=tf.matmul(a_matrixG,abgm_helperG)
    all_eigsG=tf.stack([tf.linalg.eigvalsh(a_matrixG),tf.linalg.eigvalsh(b_matrixG),tf.linalg.eigvalsh(a_b_geometric_meanG)])
    if(to_print):
      print(sess.run(all_eigsG))
    else:
      sess.run(all_eigsG)

gpu(2,eps_default,True)

# Runs the op several times.
my_number=10
print('Time (s) to compute eigenvalues of A,B and A#B with A and B being %d by %d. Average of %d runs.'%(matrix_dim,matrix_dim,my_number))
cpu_time = timeit.timeit('cpu()', number=my_number, setup="from __main__ import cpu")
print('CPU (s): %.5f'%(cpu_time/my_number))
gpu_time = timeit.timeit('gpu()', number=my_number, setup="from __main__ import gpu")
print('GPU (s): %.5f'%(gpu_time/my_number))
print('GPU vs CPU: %.5f'%(cpu_time/gpu_time))

sess.close()

[[ 1.0197163e-03  4.8639803e+00]
 [ 3.3542997e-01  2.9842005e+00]
 [-5.1122528e-01  1.7666117e+00]]
Time (s) to compute eigenvalues of A,B and A#B with A and B being 500 by 500. Average of 10 runs.
CPU (s): 0.24732
GPU (s): 1.06775
GPU vs CPU: 0.23163
